# import

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
from sklearn import preprocessing

In [101]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.svm import LinearSVC

In [20]:
import warnings

In [74]:
from collections import namedtuple

In [158]:
from imblearn.over_sampling import SMOTE

# 1. Multi-class and Multi-Label Classification Using Support Vector Machines

## (a) Pre-processing

In [5]:
# read the data
frogs_rawdata = pd.read_csv('./1 Multi-class and Multi-Label Classification Using Support Vector Machines/Anuran Calls (MFCCs)/Frogs_MFCCs.csv').drop(["RecordID"], axis = 1)
frogs_columns = list(frogs_rawdata)
frogs_rawdata.head()


,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre


In [205]:
# split the data to X and y
frogs_X_rawdata = frogs_rawdata.iloc[:, 0:22]
frogs_y = frogs_rawdata.iloc[:, 22:25]

# split the X and y to their training and test sets
frogs_X_train_rawdata, frogs_X_test_rawdata, frogs_y_train, frogs_y_test = train_test_split(frogs_X_rawdata, frogs_y, test_size=0.3, random_state=42)
frogs_y_train = frogs_y_train.reset_index(drop = True)
frogs_y_test = frogs_y_test.reset_index(drop = True)

# split y_train and y_test in addition
frogs_y_f_train = pd.DataFrame(data = frogs_y_train.iloc[:, 0], index =None, columns=['Family'])
print(frogs_y_f_train.head(), '\n')
frogs_y_g_train = pd.DataFrame(data = frogs_y_train.iloc[:, 1], index =None, columns=['Genus'])
print(frogs_y_g_train.head(), '\n')
frogs_y_s_train = pd.DataFrame(data = frogs_y_train.iloc[:, 2], index =None, columns=['Species'])
print(frogs_y_s_train.head(), '\n')

frogs_y_f_test = pd.DataFrame(data = frogs_y_test.iloc[:, 0], index =None, columns=['Family'])
print(frogs_y_f_test.head(), '\n')
frogs_y_g_test = pd.DataFrame(data = frogs_y_test.iloc[:, 1], index =None, columns=['Genus'])
print(frogs_y_g_test.head(), '\n')
frogs_y_s_test = pd.DataFrame(data = frogs_y_test.iloc[:, 2], index =None, columns=['Species'])
print(frogs_y_s_test.head(), '\n')


            Family
0  Leptodactylidae
1    Dendrobatidae
2          Hylidae
3  Leptodactylidae
4  Leptodactylidae 

       Genus
0  Adenomera
1   Ameerega
2  Hypsiboas
3  Adenomera
4  Adenomera 

                  Species
0  AdenomeraHylaedactylus
1      Ameeregatrivittata
2    HypsiboasCinerascens
3  AdenomeraHylaedactylus
4          AdenomeraAndre 

            Family
0  Leptodactylidae
1          Hylidae
2          Hylidae
3  Leptodactylidae
4  Leptodactylidae 

       Genus
0  Adenomera
1  Hypsiboas
2  Hypsiboas
3  Adenomera
4  Adenomera 

                  Species
0  AdenomeraHylaedactylus
1    HypsiboasCinerascens
2       HypsiboasCordobae
3  AdenomeraHylaedactylus
4  AdenomeraHylaedactylus 



In [7]:
# standardize the Xs

scaler = preprocessing.StandardScaler().fit(frogs_X_train_rawdata)
frogs_X_train = pd.DataFrame(data = scaler.transform(frogs_X_train_rawdata),
                             index = None,
                             columns = frogs_columns[0:22])
print(frogs_X_train.head(), '\n')
print('The size if standardized training set of independent variables is', frogs_X_train.shape, '\n')


frogs_X_test = pd.DataFrame(data = scaler.transform(frogs_X_test_rawdata),
                             index = None,
                             columns = frogs_columns[0:22])
print(frogs_X_test.head(),'\n')
print('The size if standardized test set of independent variables is', frogs_X_test.shape)

   MFCCs_ 1  MFCCs_ 2  MFCCs_ 3  MFCCs_ 4  MFCCs_ 5  MFCCs_ 6  MFCCs_ 7  \
0  0.152650  0.003519 -0.113627  0.784358 -0.200077 -1.106064 -0.460771   
1  0.152650 -0.425788 -1.730997 -3.300931 -1.247680  1.314366  2.713662   
2 -0.303474  0.586027  2.605770  0.592614 -3.358105  1.742951  1.435300   
3  0.152650 -0.312156 -0.756501  0.504547  0.355322 -0.439906 -0.946771   
4  0.152650  0.340871  0.950464  0.822589 -1.582777 -1.288530  2.603894   

   MFCCs_ 8  MFCCs_ 9  MFCCs_10    ...     MFCCs_13  MFCCs_14  MFCCs_15  \
0  1.257761  0.937894 -1.401044    ...     1.392737 -0.718716 -0.844945   
1  2.243005 -1.689411 -2.671442    ...    -1.256970  2.445364  2.011597   
2 -0.555779 -0.345921 -0.860185    ...    -1.085333 -0.669404  1.415884   
3 -0.126135  0.369278 -0.295261    ...     1.229407 -0.236187 -1.168372   
4  0.570650 -2.549861  0.786700    ...    -1.372063  1.524571  1.223935   

   MFCCs_16  MFCCs_17  MFCCs_18  MFCCs_19  MFCCs_20  MFCCs_21  MFCCs_22  
0  1.149412  0.938437 -0

## (b) Solve a multi-class and multi-label problem by training a classifier for each label.

### (i) Research

Exact match
Exact match is a multi-classification model evaluation approach that calculate the accuracy of predictions that exactly match the true labels in each instance.


Hamming loss
Exact match is a multi-classification model evaluation approach that calculate the error rate of any single predicted label.


### (ii) Train a SVM model with Gaussian kernels and one versus all classifiers determined by 10 folds cross validation

In [95]:
# for label"Family"

frogs_cv_G_f_dict= {}
frogs_para = namedtuple("frogs_para", ["C", "gamma"])
for c in (1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3):
    for gm in np.arange(0.1, 2.1, 0.1):
        clf_G_f = svm.SVC(kernel='rbf', C = c, gamma = gm) 
        # 10 folds cross validation on each paired parameters
        clf_G_f_cv_scores = cross_val_score(clf_G_f, frogs_X_train, frogs_y_f_train, cv = 10)
        clf_G_f_cv = sum(clf_G_f_cv_scores)/10
        # add cv scores to the dictionary
        frogs_cv_G_f_dict[clf_G_f_cv] = frogs_para(C = c, gamma = gm)
print(frogs_cv_G_f_dict)

{0.9910646884613632: frogs_para(C=10.0, gamma=0.1), 0.9851044120964954: frogs_para(C=10.0, gamma=0.2), 0.9809333970207215: frogs_para(C=10.0, gamma=0.30000000000000004), 0.9761698910280986: frogs_para(C=10.0, gamma=0.4), 0.9712016797175067: frogs_para(C=10.0, gamma=0.5), 0.9666377713521819: frogs_para(C=10.0, gamma=0.6), 0.964054851467637: frogs_para(C=10.0, gamma=0.7000000000000001), 0.9642556262312819: frogs_para(C=10.0, gamma=0.8), 0.9289120130910884: frogs_para(C=10.0, gamma=0.9), 0.9112406072053313: frogs_para(C=10.0, gamma=1.0), 0.8971469287779327: frogs_para(C=10.0, gamma=1.1), 0.8820552886790353: frogs_para(C=10.0, gamma=1.2000000000000002), 0.8663676184617091: frogs_para(C=10.0, gamma=1.3000000000000003), 0.8518688606324052: frogs_para(C=10.0, gamma=1.4000000000000001), 0.8417411130799805: frogs_para(C=10.0, gamma=1.5000000000000002), 0.8284395216345166: frogs_para(C=10.0, gamma=1.6), 0.8181102057145196: frogs_para(C=10.0, gamma=1.7000000000000002), 0.807387600484114: frogs_pa

In [96]:
# determine the parameters
frogs_para_G_f = max(zip(frogs_cv_G_f_dict.keys(), frogs_cv_G_f_dict.values()))
print('The highest cross-validation score and its parameters for "Family" is', frogs_para_G_f)


The highest cross-validation score and its parameters for "Family" is (0.9910646884613632, frogs_para(C=10.0, gamma=0.1))


In [181]:
# for label "Genus"

frogs_cv_G_g_dict= {}
frogs_para = namedtuple("frogs_para", ["C", "gamma"])
for c in (1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3):
    for gm in np.arange(0.1, 2.1, 0.1):
        clf_G_g = svm.SVC(kernel='rbf', C = c, gamma = gm)
        # 10 folds cross validation on each paired parameters
        clf_G_g_cv_scores = cross_val_score(clf_G_g, frogs_X_train, frogs_y_g_train, cv = 10)
        clf_G_g_cv = sum(clf_G_g_cv_scores)/10
        # add cv scores to the dictionary
        frogs_cv_G_g_dict[clf_G_g_cv] = frogs_para(C = c, gamma = gm)
print(frogs_cv_G_g_dict)

{0.9874880145443072: frogs_para(C=10.0, gamma=0.1), 0.9771660808006718: frogs_para(C=10.0, gamma=0.2), 0.9684247832426021: frogs_para(C=10.0, gamma=0.30000000000000004), 0.9606743892898633: frogs_para(C=10.0, gamma=0.4), 0.9398254641696978: frogs_para(C=10.0, gamma=0.5), 0.9249222669859156: frogs_para(C=10.0, gamma=0.6), 0.9066542706355356: frogs_para(C=10.0, gamma=0.7000000000000001), 0.8893740299398412: frogs_para(C=10.0, gamma=0.8), 0.8762742621025401: frogs_para(C=10.0, gamma=0.9), 0.859404671687703: frogs_para(C=10.0, gamma=1.0), 0.8447124555444623: frogs_para(C=10.0, gamma=1.1), 0.831222958103006: frogs_para(C=10.0, gamma=1.2000000000000002), 0.8181059072043219: frogs_para(C=10.0, gamma=1.3000000000000003), 0.8044156365057112: frogs_para(C=10.0, gamma=1.4000000000000001), 0.7938858337029153: frogs_para(C=10.0, gamma=1.5000000000000002), 0.784165055120249: frogs_para(C=10.0, gamma=1.6), 0.7730407985741644: frogs_para(C=10.0, gamma=1.7000000000000002), 0.7649002693618727: frogs_par

In [182]:
# determine the parameters
frogs_para_G_g = max(zip(frogs_cv_G_g_dict.keys(), frogs_cv_G_g_dict.values()))
print('The highest cross-validation score and its parameters for "Genus" is', frogs_para_G_g)


The highest cross-validation score and its parameters for "Genus" is (0.9874880145443072, frogs_para(C=10.0, gamma=0.1))


In [183]:
# for label "Species"

frogs_cv_G_s_dict= {}
frogs_para = namedtuple("frogs_para", ["C", "gamma"])
for c in (1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3):
    for gm in np.arange(0.1, 2.1, 0.1):
        clf_G_s = svm.SVC(kernel='rbf', C = c, gamma = gm)
        # 10 folds cross validation on each paired parameters
        clf_G_s_cv_scores = cross_val_score(clf_G_s, frogs_X_train, frogs_y_s_train, cv = 10)
        clf_G_s_cv = sum(clf_G_s_cv_scores)/10
        # add cv scores to the dictionary
        frogs_cv_G_s_dict[clf_G_s_cv] = frogs_para(C = c, gamma = gm)
print(frogs_cv_G_s_dict)

{0.9847025346338063: frogs_para(C=100.0, gamma=0.1), 0.9737839821654081: frogs_para(C=10.0, gamma=0.2), 0.96563589535558: frogs_para(C=10.0, gamma=0.30000000000000004), 0.9483665147461874: frogs_para(C=10.0, gamma=0.4), 0.9332698824034231: frogs_para(C=10.0, gamma=0.5), 0.9151958117187956: frogs_para(C=10.0, gamma=0.6), 0.8961205387801607: frogs_para(C=10.0, gamma=0.7000000000000001), 0.8786549685581433: frogs_para(C=10.0, gamma=0.8), 0.8581991001744698: frogs_para(C=10.0, gamma=0.9), 0.8389506331394816: frogs_para(C=10.0, gamma=1.0), 0.8191002560537379: frogs_para(C=10.0, gamma=1.1), 0.8014229248394527: frogs_para(C=10.0, gamma=1.2000000000000002), 0.7835534801015607: frogs_para(C=10.0, gamma=1.3000000000000003), 0.76686906069033: frogs_para(C=10.0, gamma=1.4000000000000001), 0.7517972194140605: frogs_para(C=10.0, gamma=1.5000000000000002), 0.7372981857732952: frogs_para(C=10.0, gamma=1.6), 0.7202291400413767: frogs_para(C=10.0, gamma=1.7000000000000002), 0.7077174139766688: frogs_par

In [184]:
# determine the parameters
frogs_para_G_s = max(zip(frogs_cv_G_s_dict.keys(), frogs_cv_G_s_dict.values()))
print('The highest cross-validation score and its parameters for "Species" is', frogs_para_G_s)


The highest cross-validation score and its parameters for "Species" is (0.9847041219353935, frogs_para(C=10.0, gamma=0.1))


In [185]:
# apply the classifiers to all 3 labels on test set

clf_G_f.fit(frogs_X_train, frogs_y_f_train)
frogs_y_f_test_G_pred = pd.DataFrame(data = clf_G_f.predict(frogs_X_test), columns = [frogs_columns[-3]], index=None)

clf_G_g.fit(frogs_X_train, frogs_y_g_train)
frogs_y_g_test_G_pred = pd.DataFrame(data = clf_G_g.predict(frogs_X_test), columns = [frogs_columns[-2]], index=None)

clf_G_s.fit(frogs_X_train, frogs_y_s_train)
frogs_y_s_test_G_pred = pd.DataFrame(data = clf_G_s.predict(frogs_X_test), columns = [frogs_columns[-1]], index=None)


In [186]:
frogs_y_test_G_pred = frogs_y_f_test_G_pred.merge(frogs_y_g_test_G_pred, how = 'inner',left_index=True, right_index=True).merge(frogs_y_s_test_G_pred, how = 'inner',left_index=True, right_index=True)
frogs_y_test_G_pred

,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
2,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
3,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
4,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
5,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
6,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
7,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
8,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
9,Leptodactylidae,Adenomera,AdenomeraHylaedactylus


In [189]:
# exact match

def em_accuracy_score(y_true, y_pred):
    matched = 0
    for y_p, y_t in zip(y_pred, y_true):
        if y_t in y_p:
            matched = matched + 1
    return (matched / len(y_true))


print("The exact match of test data is",
      em_accuracy_score(frogs_y_test.values, frogs_y_test_G_pred.values))

The exact match of test data is 0.6238999536822603


In [190]:
# hamming loss

frogs_G_f_hl = hamming_loss(frogs_y_f_test, frogs_y_f_test_G_pred)
frogs_G_g_hl = hamming_loss(frogs_y_g_test, frogs_y_g_test_G_pred)
frogs_G_s_hl = hamming_loss(frogs_y_s_test, frogs_y_s_test_G_pred)

frogs_G_hl = (frogs_G_f_hl + frogs_G_g_hl + frogs_G_s_hl)/3

print("The hamming loss of test data is", frogs_G_hl)

The hamming loss of test data is 0.43970974216458236


### (iii) Train a SVM model with L1-penalized kernels(L1 penalty with linear kernels) and one versus all classifiers determined by 10 folds cross validation

In [191]:
# for label"Family"

frogs_cv_L_f_dict= {}
frogs_para_L = namedtuple("frogs_para_L", ["C"])
for c in (1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3):
    clf_L_f = LinearSVC(penalty = 'l1', loss='squared_hinge', dual=False) 
    # 10 folds cross validation on each paired parameters
    clf_L_f_cv_scores = cross_val_score(clf_L_f, frogs_X_train, frogs_y_f_train, cv = 10)
    clf_L_f_cv = sum(clf_L_f_cv_scores)/10
    # add cv scores to the dictionary
    frogs_cv_L_f_dict[clf_L_f_cv] = frogs_para_L(C = c)
print(frogs_cv_L_f_dict)

{0.940632187068774: frogs_para_L(C=0.001)}


In [192]:
# determine the parameters
frogs_para_L_f = max(zip(frogs_cv_L_f_dict.keys(), frogs_cv_L_f_dict.values()))
print('The highest cross-validation score and its parameters for "Family" is', frogs_para_L_f)


The highest cross-validation score and its parameters for "Family" is (0.940632187068774, frogs_para_L(C=0.001))


In [193]:
# for label"Genus"

frogs_cv_L_g_dict= {}
frogs_para_L = namedtuple("frogs_para_L", ["C"])
for c in (1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3):
    clf_L_g = LinearSVC(penalty = 'l1', loss='squared_hinge', dual=False)
    # 10 folds cross validation on each paired parameters
    clf_L_g_cv_scores = cross_val_score(clf_L_g, frogs_X_train, frogs_y_g_train, cv = 10)
    clf_L_g_cv = sum(clf_L_g_cv_scores)/10
    # add cv scores to the dictionary
    frogs_cv_L_g_dict[clf_L_g_cv] = frogs_para_L(C = c)
print(frogs_cv_L_g_dict)

{0.9513547876190283: frogs_para_L(C=0.01), 0.9511551868206249: frogs_para_L(C=0.001), 0.9513543900078758: frogs_para_L(C=0.1)}


In [194]:
# determine the parameters
frogs_para_L_g = max(zip(frogs_cv_L_g_dict.keys(), frogs_cv_L_g_dict.values()))
print('The highest cross-validation score and its parameters for "Genus" is', frogs_para_L_g)


The highest cross-validation score and its parameters for "Genus" is (0.9513547876190283, frogs_para_L(C=0.01))


In [195]:
# for label"Species"

frogs_cv_L_s_dict= {}
frogs_para_L = namedtuple("frogs_para_L", ["C"])
for c in (1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3):
    clf_L_s = LinearSVC(penalty = 'l1', loss='squared_hinge', dual=False) 
    # 10 folds cross validation on each paired parameters
    clf_L_s_cv_scores = cross_val_score(clf_L_s, frogs_X_train, frogs_y_s_train, cv = 10)
    clf_L_s_cv = sum(clf_L_s_cv_scores)/10
    # add cv scores to the dictionary
    frogs_cv_L_s_dict[clf_L_s_cv] = frogs_para_L(C = c)
print(frogs_cv_L_s_dict)

{0.9600837758137208: frogs_para_L(C=1000.0), 0.9604837758137206: frogs_para_L(C=10.0), 0.9604837758137208: frogs_para_L(C=1.0), 0.9600837758137206: frogs_para_L(C=0.1), 0.9602837758137207: frogs_para_L(C=0.01), 0.9598861473552228: frogs_para_L(C=0.001)}


In [196]:
# determine the parameters
frogs_para_L_s = max(zip(frogs_cv_L_s_dict.keys(), frogs_cv_L_s_dict.values()))
print('The highest cross-validation score and its parameters for "Species" is', frogs_para_L_s)


The highest cross-validation score and its parameters for "Species" is (0.9604837758137208, frogs_para_L(C=1.0))


In [197]:
# apply the classifiers to all 3 labels on test set

clf_L_f.fit(frogs_X_train, frogs_y_f_train)
frogs_y_f_test_L_pred = pd.DataFrame(data = clf_L_f.predict(frogs_X_test), columns = [frogs_columns[-3]], index=None)

clf_L_g.fit(frogs_X_train, frogs_y_g_train)
frogs_y_g_test_L_pred = pd.DataFrame(data = clf_L_g.predict(frogs_X_test), columns = [frogs_columns[-2]], index=None)

clf_L_s.fit(frogs_X_train, frogs_y_s_train)
frogs_y_s_test_L_pred = pd.DataFrame(data = clf_L_s.predict(frogs_X_test), columns = [frogs_columns[-1]], index=None) 

frogs_y_test_L_pred = frogs_y_f_test_L_pred.merge(frogs_y_g_test_L_pred, how = 'inner',left_index=True, right_index=True).merge(frogs_y_s_test_L_pred, how = 'inner',left_index=True, right_index=True)
frogs_y_test_L_pred


,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCinerascens
2,Hylidae,Hypsiboas,HypsiboasCordobae
3,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
4,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
5,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
6,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
7,Leptodactylidae,Adenomera,AdenomeraAndre
8,Leptodactylidae,Adenomera,AdenomeraAndre
9,Hylidae,Hypsiboas,HypsiboasCinerascens


In [198]:
# exact match

print("The exact match of test data is",
      em_accuracy_score(frogs_y_test.values, frogs_y_test_L_pred.values))


The exact match of test data is 0.9689671144048171


In [199]:
# hamming loss

frogs_L_f_hl = hamming_loss(frogs_y_f_test, frogs_y_f_test_L_pred)
frogs_L_g_hl = hamming_loss(frogs_y_g_test, frogs_y_g_test_L_pred)
frogs_L_s_hl = hamming_loss(frogs_y_s_test, frogs_y_s_test_L_pred)

frogs_L_hl = (frogs_L_f_hl + frogs_L_g_hl + frogs_L_s_hl)/3
print("The hamming loss of test data is", frogs_L_hl)

The hamming loss of test data is 0.05681642735834491


### (iv) (balance the data by SMOTE)Train a SVM model with L1-penalized kernels(L1 penalty with linear kernels) and one versus all classifiers determined by 10 folds cross validation

In [200]:
sm = SMOTE()
frogs_X_f_train_cps, frogs_y_f_train_cps = sm.fit_resample(frogs_X_train, frogs_y_f_train)
frogs_X_g_train_cps, frogs_y_g_train_cps = sm.fit_resample(frogs_X_train, frogs_y_g_train)
frogs_X_s_train_cps, frogs_y_s_train_cps = sm.fit_resample(frogs_X_train, frogs_y_s_train)


In [201]:
# for label"Family"

frogs_cv_L_f_cps_dict= {}
frogs_para_L_cps = namedtuple("frogs_para_L_cps", ["C"])
for c in (1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3):
    clf_L_f_cps = LinearSVC(penalty = 'l1', loss='squared_hinge', dual=False) 
    # 10 folds cross validation on each paired parameters
    clf_L_f_cps_cv_scores = cross_val_score(clf_L_f_cps, frogs_X_f_train_cps, frogs_y_f_train_cps, cv = 10)
    clf_L_f_cps_cv = sum(clf_L_f_cps_cv_scores)/10
    # add cv scores to the dictionary
    frogs_cv_L_f_cps_dict[clf_L_f_cps_cv] = frogs_para_L_cps(C = c)
print(frogs_cv_L_f_cps_dict)


{0.94866824950294: frogs_para_L_cps(C=0.01), 0.9485868162781843: frogs_para_L_cps(C=0.001), 0.9485053830534286: frogs_para_L_cps(C=10.0)}


In [202]:
# determine the parameters
frogs_para_L_f_cps = max(zip(frogs_cv_L_f_cps_dict.keys(), frogs_cv_L_f_cps_dict.values()))
print('The highest cross-validation score and its parameters for "Family" is', frogs_para_L_f_cps)


The highest cross-validation score and its parameters for "Family" is (0.94866824950294, frogs_para_L_cps(C=0.01))


In [203]:
# for label"Genus"

frogs_cv_L_g_cps_dict= {}
frogs_para_L_cps = namedtuple("frogs_para_L_cps", ["C"])
for c in (1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3):
    clf_L_g_cps = LinearSVC(penalty = 'l1', loss='squared_hinge', dual=False)
    # 10 folds cross validation on each paired parameters
    clf_L_g_cps_cv_scores = cross_val_score(clf_L_g_cps, frogs_X_g_train_cps, frogs_y_g_train_cps, cv = 10)
    clf_L_g_cps_cv = sum(clf_L_g_cps_cv_scores)/10
    # add cv scores to the dictionary
    frogs_cv_L_g_cps_dict[clf_L_g_cps_cv] = frogs_para_L_cps(C = c)
print(frogs_cv_L_g_cps_dict)


{0.9567083283617708: frogs_para_L_cps(C=1000.0), 0.9568377878534783: frogs_para_L_cps(C=100.0), 0.9567084775086506: frogs_para_L_cps(C=0.01), 0.9566222706120989: frogs_para_L_cps(C=0.1), 0.9565791671638231: frogs_para_L_cps(C=0.001)}


In [167]:
# determine the parameters
frogs_para_L_g_cps = max(zip(frogs_cv_L_g_cps_dict.keys(), frogs_cv_L_g_cps_dict.values()))
print('The highest cross-validation score and its parameters for "Genus" is', frogs_para_L_g_cps)


The highest cross-validation score and its parameters for "Genus" is (0.9590837907170983, frogs_para_L_cps(C=1.0))


In [176]:
# for label"Species"

frogs_cv_L_s_cps_dict= {}
frogs_para_L_cps = namedtuple("frogs_para_L_cps", ["C"])
for c in (1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3):
    clf_L_s_cps = LinearSVC(penalty = 'l1', loss='squared_hinge', dual=False) 
    # 10 folds cross validation on each paired parameters
    clf_L_s_cps_cv_scores = cross_val_score(clf_L_s_cps, frogs_X_s_train_cps, frogs_y_s_train_cps, cv = 10)
    clf_L_s_cps_cv = sum(clf_L_s_cps_cv_scores)/10
    # add cv scores to the dictionary
    frogs_cv_L_s_cps_dict[clf_L_s_cps_cv] = frogs_para_L_cps(C = c)
print(frogs_cv_L_s_cps_dict)


{0.9622793576446973: frogs_para_L_cps(C=0.1), 0.9622385413181667: frogs_para_L_cps(C=1.0), 0.962156574105052: frogs_para_L_cps(C=0.01), 0.962115925058548: frogs_para_L_cps(C=0.001)}


In [177]:
# determine the parameters
frogs_para_L_s_cps = max(zip(frogs_cv_L_s_cps_dict.keys(), frogs_cv_L_s_cps_dict.values()))
print('The highest cross-validation score and its parameters for "Species" is', frogs_para_L_s_cps)


The highest cross-validation score and its parameters for "Species" is (0.9622793576446973, frogs_para_L_cps(C=0.1))


In [178]:
# apply the classifiers to all 3 labels on test set

clf_L_f_cps.fit(frogs_X_f_train_cps, frogs_y_f_train_cps)
frogs_y_f_test_cps_L_pred = pd.DataFrame(data = clf_L_f_cps.predict(frogs_X_test), columns = [frogs_columns[-3]], index=None)

clf_L_g_cps.fit(frogs_X_g_train_cps, frogs_y_g_train_cps)
frogs_y_g_test_cps_L_pred = pd.DataFrame(data = clf_L_g_cps.predict(frogs_X_test), columns = [frogs_columns[-2]], index=None)

clf_L_s_cps.fit(frogs_X_s_train_cps, frogs_y_s_train_cps)
frogs_y_s_test_cps_L_pred = pd.DataFrame(data = clf_L_s_cps.predict(frogs_X_test), columns = [frogs_columns[-1]], index=None) 

frogs_y_test_cps_L_pred = frogs_y_f_test_cps_L_pred.merge(frogs_y_g_test_cps_L_pred, how = 'inner',left_index=True, right_index=True).merge(frogs_y_s_test_cps_L_pred, how = 'inner',left_index=True, right_index=True)
frogs_y_test_cps_L_pred


,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCinerascens
2,Hylidae,Hypsiboas,HypsiboasCordobae
3,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
4,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
5,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
6,Leptodactylidae,Dendropsophus,AdenomeraHylaedactylus
7,Leptodactylidae,Adenomera,AdenomeraAndre
8,Leptodactylidae,Adenomera,AdenomeraAndre
9,Hylidae,Scinax,HypsiboasCinerascens


In [179]:
# exact match

print("The exact match of test data is",
      em_accuracy_score(frogs_y_test.values, frogs_y_test_cps_L_pred.values))


The exact match of test data is 0.9745252431681334


In [180]:
# hamming loss

frogs_L_f_cps_hl = hamming_loss(frogs_y_f_test, frogs_y_f_test_cps_L_pred)
frogs_L_g_cps_hl = hamming_loss(frogs_y_g_test, frogs_y_g_test_cps_L_pred)
frogs_L_s_cps_hl = hamming_loss(frogs_y_s_test, frogs_y_s_test_cps_L_pred)

frogs_L_cps_hl = (frogs_L_f_cps_hl + frogs_L_g_cps_hl + frogs_L_s_cps_hl)/3
print("The hamming loss of test data is", frogs_L_cps_hl)

The hamming loss of test data is 0.07735062528948587


# 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set Monte-Carlo Simulation

## (a) Choose best k

In [209]:
from sklearn.cluster import KMeans
from sklearn import metrics


k_range = range(2, 51)
best_clustering_score = {'ch_score': -np.inf,
                         'silhouette_score': -np.inf,
                         'db_score': np.inf}
best_clustering_k = {'ch_score': np.nan,
                     'silhouette_score': np.nan,
                     'db_score': np.nan}

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(frogs_X_rawdata)
    labels = kmeans.labels_

    # different methods to evaluate clustering
    ch_score = metrics.calinski_harabaz_score(frogs_X_rawdata, labels)
    silhouette_score = metrics.silhouette_score(frogs_X_rawdata, labels, metric='euclidean')
    db_score = metrics.davies_bouldin_score(frogs_X_rawdata, labels)
    
    # higher calinski harabaz score and silhouette score is better
    if ch_score > best_clustering_score['ch_score']:
        best_clustering_score['ch_score'] = ch_score
        best_clustering_k['ch_score'] = k
    
    if silhouette_score > best_clustering_score['silhouette_score']:
        best_clustering_score['silhouette_score'] = silhouette_score
        best_clustering_k['silhouette_score'] = k
    
    # lower davies bouldin score is better
    if db_score < best_clustering_score['db_score']:
        best_clustering_score['db_score'] = db_score
        best_clustering_k['db_score'] = k

        
print("The best k choosed by different methods:")
print("The best k is", best_clustering_k)
print("The best k' scores are", best_clustering_score)



The best k choosed by different methods:
The best k is {'ch_score': 2, 'silhouette_score': 4, 'db_score': 5}
The best k' scores are {'ch_score': 3680.7413494630937, 'silhouette_score': 0.3788851472047738, 'db_score': 1.2285513618884898}


## (b) The Clustering's Label for the best k by Silhouette Score

In [215]:
kmeans = KMeans(n_clusters=best_clustering_k['silhouette_score'], random_state=0).fit(frogs_X_rawdata)
clusters = kmeans.labels_

pred_y_cluster = frogs_y.copy()
for i in set(clusters):
    print('For', i+1, 'cluster,')
    for label in frogs_columns[-3:]:
        print('The majorit label for', label, 'is', frogs_y[label][clusters == i].mode()[0])
        pred_y_cluster[label][clusters == i] = frogs_y[label][clusters == i].mode()[0]

For 1 cluster,
The majorit label for Family is Leptodactylidae
The majorit label for Genus is Adenomera
The majorit label for Species is AdenomeraHylaedactylus
For 2 cluster,
The majorit label for Family is Dendrobatidae
The majorit label for Genus is Ameerega
The majorit label for Species is Ameeregatrivittata
For 3 cluster,
The majorit label for Family is Hylidae
The majorit label for Genus is Hypsiboas
The majorit label for Species is HypsiboasCinerascens
For 4 cluster,
The majorit label for Family is Hylidae
The majorit label for Genus is Hypsiboas
The majorit label for Species is HypsiboasCordobae


## (c) Hamming Distance between True Labels and the Labels Assigned by Clusters

In [217]:
print("The hamming score of whose data between true labels and the labels assigned by clusters\n",
      np.array([hamming_loss(frogs_y.values[:, i], pred_y_cluster.values[:, i]) for i in range(3)]).mean())


The hamming score of whose data between true labels and the labels assigned by clusters
 0.22191336576326157
